1.1:

In [16]:
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold, LeaveOneOut, cross_val_score, GridSearchCV, ParameterGrid
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import confusion_matrix
# import category_encoders as ce
import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline

In [17]:
from sklearn.datasets import fetch_openml
credit_data = fetch_openml("credit-g", as_frame=True)

/Users/umesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/datasets/_openml.py:417: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  warn(


1.1) 
The continous features are duration, credit_amount, installment_commitment, residence_since, age, existing_credits, num_dependents. The categorical features are class(target), checking_status, credit_history, purpose, saving_status, employment, personal_status, other_parties, property_magnitude, other_payment_plans, housing, job, own_telephone, foreign_worker. 

In [11]:
credit_data.feature_names

['checking_status',
 'duration',
 'credit_history',
 'purpose',
 'credit_amount',
 'savings_status',
 'employment',
 'installment_commitment',
 'personal_status',
 'other_parties',
 'residence_since',
 'property_magnitude',
 'age',
 'other_payment_plans',
 'housing',
 'existing_credits',
 'job',
 'num_dependents',
 'own_telephone',
 'foreign_worker']

In [12]:
credit_data.categories

Categorical features contain finite number of distinct groups. Going by that definition and looking at the results above, following features can be classified as being categorical:

checking_status
credit_history
employment
foreign_worker
housing
job
other_parties
other_payment_plans
own_telephone
personal_status
property_magnitude
purpose
savings_status
Continuous features can have an infinite range of values. From the above list of features, we can say that the following features are continuous:

duration
credit_amount
installment_commitment (installment rate in percentage of disposable income)
residence_since (number of years)
age
existing_credits
num_dependents
All the features in the above list are numeric.

1.2:

We ordinal encode all the categorical features before creating the training, validation and test splits. We then scale the continuous features. Converting the data bunch to a data frame seems to have ordinally encoded the categorical variables, but to be on the safe side the following steps use 'as_frame=True' to download the dataset as a dataframe. We then perform an ordinal encoding on the categorical variables.

In [23]:
credit_dataX, credit_datay = fetch_openml('credit-g', version='active', as_frame=True, return_X_y =True)
categorical_features = ['checking_status', 'credit_history', 'employment', 'foreign_worker',
'housing',
'job',
'other_parties',
'other_payment_plans',
'own_telephone',
'personal_status',
'property_magnitude',
'purpose',
'savings_status']
credit_dataX_ordinal = credit_dataX.copy()
for feature in credit_dataX.iteritems():
  if feature[0] in categorical_features:
    credit_dataX_ordinal[feature[0]] = credit_dataX[feature[0]].astype("category").cat.codes

/Users/umesh/opt/anaconda3/lib/python3.9/site-packages/sklearn/datasets/_openml.py:417: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  warn(


In [24]:
credit_dataX_ordinal.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,0,6.0,4,3,1169.0,4,4,4.0,2,0,4.0,0,67.0,2,1,2.0,2,1.0,1,0
1,1,48.0,2,3,5951.0,0,2,2.0,1,0,2.0,0,22.0,2,1,1.0,2,1.0,0,0
2,3,12.0,4,6,2096.0,0,3,2.0,2,0,3.0,0,49.0,2,1,1.0,1,2.0,0,0
3,0,42.0,2,2,7882.0,0,3,2.0,2,2,4.0,1,45.0,2,2,1.0,2,2.0,0,0
4,0,24.0,3,0,4870.0,0,2,3.0,2,0,4.0,3,53.0,2,2,2.0,2,2.0,0,0


Performing Label encoding on the target

In [25]:
le = LabelEncoder()
y = le.fit_transform(credit_datay)

Creating the training, validation and test splits

In [26]:
# creating the training test split
X_trainval, X_test, y_trainval, y_test = train_test_split(credit_dataX_ordinal, y)
# creating the training validation split
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval)

In [27]:
numeric_features = [feature for feature in credit_data.feature_names if feature not in categorical_features]

In [28]:
scaler = StandardScaler()
X_train_scaled = X_train
X_train_scaled[numeric_features] = scaler.fit_transform(X_train_scaled[numeric_features])

X_val_scaled = X_val
X_val_scaled[numeric_features] = scaler.fit_transform(X_val_scaled[numeric_features])

logistic_regression = LogisticRegression().fit(X_train, y_train)
logistic_regression.predict(X_val)
logistic_regression.score(X_val, y_val)

0.8031914893617021

1.4:

In [29]:
# creating the training test split anew
X_trainval_pipe, X_test_pipe, y_trainval_pipe, y_test_pipe = train_test_split(credit_dataX, credit_datay)
# creating a Repeated Stratified K fold
skf = StratifiedKFold(n_splits=5, shuffle=True)

Logistic Regression (without scaled data)

In [30]:
preprocess_lr = make_column_transformer(
    (OneHotEncoder(), categorical_features))
model_lr = make_pipeline(preprocess_lr, LogisticRegression())
model_lr.fit(X_trainval_pipe, y_trainval_pipe)
scores_lr = cross_val_score(model_lr, X_trainval_pipe, y_trainval_pipe, cv=skf, scoring='accuracy')
print("score: {:.3f}".format(np.mean(scores_lr)))

score: 0.713


Logistic Regression (with scaling)

In [32]:
preprocess_lrs = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(), categorical_features))
model_lrs = make_pipeline(preprocess_lrs, LogisticRegression())
model_lrs.fit(X_trainval_pipe, y_trainval_pipe)
scores_lrs = cross_val_score(model_lrs, X_trainval_pipe, y_trainval_pipe, cv=skf, scoring='accuracy')
print("score: {:.3f}".format(np.mean(scores_lrs)))

score: 0.724


Linear Support Vector (SVC) without scaling

In [33]:
preprocess_svc = make_column_transformer(
    (OneHotEncoder(), categorical_features))
model_svc = make_pipeline(preprocess_svc, LinearSVC())
model_svc.fit(X_trainval_pipe, y_trainval_pipe)
scores_svc = cross_val_score(model_svc, X_trainval_pipe, y_trainval_pipe, cv=skf, scoring='accuracy')
print("score: {:.3f}".format(np.mean(scores_svc)))

score: 0.724


Linear Support Vector (SVC) with scaling

In [34]:
preprocess_svcs = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(), categorical_features))
model_svcs = make_pipeline(preprocess_svcs, LinearSVC( max_iter=5000))
model_svcs.fit(X_trainval_pipe, y_trainval_pipe)
scores_svcs = cross_val_score(model_svcs, X_trainval_pipe, y_trainval_pipe, cv=skf, scoring='accuracy')
print("score: {:.3f}".format(np.mean(scores_svcs)))

score: 0.736


Nearest neighbors without scaling

In [36]:
preprocess_knn = make_column_transformer(
    (OneHotEncoder(), categorical_features))
model_knn = make_pipeline(preprocess_knn, KNeighborsClassifier())
model_knn.fit(X_trainval_pipe, y_trainval_pipe)
scores_knn = cross_val_score(model_knn, X_trainval_pipe, y_trainval_pipe, cv=skf, scoring='accuracy')
print("score: {:.3f}".format(np.mean(scores_knn)))

score: 0.713


Nearest neighbors with scaling

In [38]:
preprocess_knns = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(), categorical_features))
model_knns = make_pipeline(preprocess_knns, KNeighborsClassifier())
model_knns.fit(X_trainval_pipe, y_trainval_pipe)
scores_knns = cross_val_score(model_knns, X_trainval_pipe, y_trainval_pipe, cv=skf, scoring='accuracy')
print("score: {:.3f}".format(np.mean(scores_knns)))

score: 0.707


Conclusion
From the results above we see that the accuracy is better when we use scaling on numeric features. For logistic regression, the accuracy improved from 0.732 to 0.745 when the data is scaled. For SVC the accuracy improved from 0.741 to 0.749 and for Nearest neighbors the accuracy improved from 0.716 to 0.738.

1.5
Using GridSearchCV

Creating a new split of data, using encoded data points for GridSearchCV